<a href="https://colab.research.google.com/github/machine-perception-robotics-group/ImageProcessingGoogleColabNotebooks/blob/master/03_geo_transform_image_mosaicking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 03. 画像の幾何変換，イメージモザイク

講義で説明する画像処理の方法について，google colaboratoryを利用して演習する．
google colaboratoryは，クラウドで実行する Jupyter ノートブック環境である.
google coraboratoryについては，[ここ](https://www.tdi.co.jp/miso/google-colaboratory-gpu)や[ここ](https://www.codexa.net/how-to-use-google-colaboratory/)を参考にすること．

下記のプログラムを実行すると，画像の幾何変換とイメージモザイクを行う．


## 準備
プログラムの動作に必要なデータをダウンロードし，zipファイルを解凍する．
`!`で始まるコマンドはPythonではなく，Linux（Ubuntu）のコマンドを実行している．

In [ ]:
!wget -q https://raw.githubusercontent.com/machine-perception-robotics-group/ImageProcessingGoogleColabNotebooks/master/image1.zip
!unzip -q image1.zip
!ls
!ls ./image1/

## 画像の読み込みと表示
必要なパッケージをインポートし，画像を表示する．


In [ ]:
import math
import cv2
import numpy as np
from matplotlib import pyplot as plt

img1 = cv2.imread('./image1/woman-g.jpg', 2)
plt.imshow(img1, cmap = "gray", clim=(0, 255))
plt.show()

## 座標変換

ここでは，画像の座標変換を行う．

### 平行移動

平行移動は，対応する画素値を指定した移動量従い，対応する配列の要素に代入することで実現される．

まず，変換後の画像データを保存するための配列を準備し，x, y方向それぞれの移動量を指定する．

その後，移動量に従い，画素値を代入することで，平行移動を行う．

In [ ]:
img = img1.copy()
height, width = img.shape

# 変換後の画像データを保存するための配列を準備
img_translated = np.ones(img.shape, dtype=np.uint8) * 255

# x, y方向それぞれの移動量を指定
t_x = 100
t_y = 10

for y in range(height):
  for x in range(width):
    if 0 < y+t_y < height and 0 < x+t_x < width:
      img_translated[y+t_y, x+t_x] = img[y, x]

plt.imshow(img_translated, cmap="gray", clim=(0, 255))

### 拡大縮小

拡大縮小では，画像座標系の原点（0, 0）を基準に指定した倍率に画像を拡大縮小する．


In [ ]:
img = img1.copy()
height, width = img.shape

# 変換後の画像データを保存するための配列を準備
img_translated = np.ones(img.shape, dtype=np.uint8) * 255

# x, y方向それぞれの拡大縮小率を指定
Sx = 1.5
Sy = 1.5

for y in range(height):
  for x in range(width):
    if 0 < y*Sy < height and 0 < x*Sx < width:
      img_translated[int(y*Sy), int(x*Sx)] = img[y, x]

plt.imshow(img_translated, cmap="gray", clim=(0, 255))

### 回転

画像の回転は，画像座標系の原点を軸として，指定した角度に画像を回転させる．

`theta`で回転角度を指定する．
その後，`math.sin`, `math.cos`関数を用いて，指定した角度に対するsin, cosを計算する．
この時，`math.sin`, `math.cos`関数へ入力する角度は弧度法に基づくため，`math.radian`関数を用いて，度数法で表現した角度を変換して入力する．

得られたsin, cosを用いて，変換した座標に画素値を代入することで，画像の回転を行う．

In [ ]:
img = img1.copy()
height, width = img.shape

# 変換後の画像データを保存するための配列を準備
img_translated = np.ones(img.shape, dtype=np.uint8) * 255

# 回転角度を指定
theta = -15
sin_th = math.sin(math.radians(theta))
cos_th = math.cos(math.radians(theta))

for y in range(height):
  for x in range(width):
    if 0 < (x*sin_th + y*cos_th) < height and 0 < (x*cos_th - y*sin_th) < width:
      img_translated[int(x*sin_th + y*cos_th), int(x*cos_th - y*sin_th)] = img[y, x]

plt.imshow(img_translated, cmap="gray", clim=(0, 255))

### スキュー

スキューでは，指定した角度に画像を傾けるような変換を行う．
以下では，水平方向・垂直方向のスキューを行う．

#### 水平方向のスキュー

In [ ]:
img = img1.copy()
height, width = img.shape

# 変換後の画像データを保存するための配列を準備
img_translated = np.ones(img.shape, dtype=np.uint8) * 255

# 水平方向のスキューの角度を指定
theta = 30
tan_th = math.tan(math.radians(theta))

for y in range(height):
    for x in range(width):
      if 0 < int(x + y*tan_th) < width:
        img_translated[y, int(x + y*tan_th)] = img[y, x]

plt.imshow(img_translated, cmap="gray", clim=(0, 255))

#### 垂直方向のスキュー

In [ ]:
img = img1.copy()
height, width = img.shape

# 変換後の画像データを保存するための配列を準備
img_translated = np.ones(img.shape, dtype=np.uint8) * 255

# 垂直方向のスキューの角度を指定
theta = 30
tan_th = math.tan(math.radians(theta))

for y in range(height):
    for x in range(width):
      if 0 < int(x*tan_th + y) < height:
        img_translated[int(x*tan_th + y), x] = img[y, x]

plt.imshow(img_translated, cmap="gray", clim=(0, 255))

### アフィン変換

アフィン変換は同次座標を導入した3x3の行列で任意の画像変換を行う方法である．

以下では，アフィン変換を用いて複数の画像変換を行う．
具体的には
1. 平行移動
2. 拡大縮小
3. 回転
4. 平行移動（1. とは逆方向）

を連続して処理する．

まず，各変換を行うためのアフィン行列（`T1`, `S`, `R`, `T2`）を定義する．
各行列はNumpyの`matrix`で定義を行う．これまでに使用していたNumpyの`array`とは異なり，掛け算等の演算を行う際には，行列演算に従った計算を行う．

In [ ]:
# 画像の中心を原点に移動するための平行移動のアフィン行列を定義
t_x = -320
t_y = -240
T1 = np.matrix([[1, 0, t_x],
                [0, 1, t_y],
                [0, 0,   1]])

# 拡大縮小のアフィン行列を定義
s_x = 0.5
s_y = 0.5
S = np.matrix([[s_x, 0, 0],
               [0, s_y, 0],
               [0,   0, 1]])

# 回転のアフィン行列を定義
theta = -45
sin_th = math.sin(math.radians(theta))
cos_th = math.cos(math.radians(theta))
R = np.matrix([[cos_th, -sin_th, 0],
               [sin_th,  cos_th, 0],
               [     0,       0, 1]])

# 変換した画像を元の座標に移動するための平行移動のアフィン行列を定義（T1とは逆方向の平行移動）
T2 = np.matrix([[1, 0, -t_x],
                [0, 1, -t_y],
                [0, 0,    1]])

#### 一つづつアフィン変換を行う方法

以下では，上で定義したアフィン行列（`T1`, `S`, `R`, `T2`）を一つづつ用いて，順番に画像変換を行う．

画像変換を順番に適用することで，任意の画像変換を行うことができる．

In [ ]:
# 変換する画像をコピー
img= img1.copy()
height, width = img.shape

# 平行移動
img_translated1 = np.ones(img.shape, dtype=np.uint8) * 255
for y in range(height):
  for x in range(width):
    x1, y1, _ = T1 * np.matrix([[x], [y], [1]])
    if 0 < int(x1) < width and 0 < int(y1) < height:
      # 元画像を平行移動する
      img_translated1[int(y1), int(x1)] = img[y, x]
  
plt.imshow(img_translated1, cmap="gray", clim=(0, 255)), plt.title("parallel translation 1")
plt.show()

# 拡大縮小
img_translated2 = np.ones(img.shape, dtype=np.uint8) * 255
for y in range(height):
  for x in range(width):
    x1, y1, _ = S * np.matrix([[x], [y], [1]])
    if 0 < int(x1) < width and 0 < int(y1) < height:
      # 平行移動した画像を拡大縮小する
      img_translated2[int(y1), int(x1)] = img_translated1[y, x]
  
plt.imshow(img_translated2, cmap="gray", clim=(0, 255)), plt.title("scaling")
plt.show()

# 回転
img_translated3 = np.ones(img.shape, dtype=np.uint8) * 255
for y in range(height):
  for x in range(width):
    x1, y1, _ = R * np.matrix([[x], [y], [1]])
    if 0 < int(x1) < width and 0 < int(y1) < height:
      # 拡大縮小した画像を回転させる
      img_translated3[int(y1), int(x1)] = img_translated2[y, x]
plt.imshow(img_translated3, cmap="gray", clim=(0, 255)), plt.title("rotation")
plt.show()

# 平行移動2
img_translated4 = np.ones(img.shape, dtype=np.uint8) * 255
for y in range(height):
  for x in range(width):
    x1, y1, _ = T2 * np.matrix([[x], [y], [1]])
    if 0 < int(x1) < width and 0 < int(y1) < height:
      # 回転した画像を平行移動する
      img_translated4[int(y1), int(x1)] = img_translated3[y, x]
plt.imshow(img_translated4, cmap="gray", clim=(0, 255)), plt.title("parallel translation 2")
plt.show()

#### 複数の変換を一つに行列で一度に行う方法

上では，アフィン変換を一つづつ適用した．

ここでは，複数のアフィン行列を一つの行列にして，変換を行う．

行列`H`に適用したい変換のアフィン行列の積を計算する．
このとき，先に適用する変換の行列を右，その後適用する変換の行列を左として積を計算することで，任意の順番の変換を行うことができる．

変換の結果を確認すると，上記と同様の手順ではあるが，こちらの方法では，画像が領域外にはみ出す影響が無く，画像が欠けることなく変換できている．

In [ ]:
# 行列をまとめる
H = T2 * R * S * T1

img_trans_all = np.ones(img.shape, dtype=np.uint8) * 255

for y in range(height):
  for x in range(width):
    x1, y1, _ = H * np.matrix([[x], [y], [1]])
    if 0 < int(x1) < width and 0 < int(y1) < height:
      img_trans_all[int(y1), int(x1)] = img[y, x]
  
plt.imshow(img_trans_all, cmap="gray", clim=(0, 255)), plt.title("affine transform (all)")
plt.show()

#### OpenCVを用いたアフィン変換

OpenCVには`warpAffine`という任意のアフィン行列を用いて画像を変換するための関数が用意されている．
以下では，`warpAffine`関数を用いたアフィン変換を行う．
`warpAffine`関数では，第1引数に変換する画像，第2引数にアフィン行列を指定する．第3引数は出力する画像のサイズを示している．

まず，上で作成したアフィン行列`H`を用いて，アフィン変換を行う．
同様の変換が実現できていることが確認できる．

また，OpenCVの`getRotationMatrix2D`関数を用いてアフィン行列を作成し，変換を行うことも可能である．
`getRotationMatrix2D`関数の詳細については割愛する．

In [ ]:
img= img1.copy()
height, width = img.shape

# 上で作成した行列Hを用いてアフィン変換
img_translated1 = cv2.warpAffine(img, H[0:2, :], (width, height))
plt.imshow(img_translated1, cmap="gray", clim=(0, 255))
plt.title("affine transform 1")
plt.show()

# OpenCVの関数を用いて変換行列を作成した場合
mat = cv2.getRotationMatrix2D((360, 240), 45, 0.5)
img_translated2 = cv2.warpAffine(img, mat, (width, height))
plt.imshow(img_translated2, cmap="gray", clim=(0, 255))
plt.title("affine transform 2")
plt.show()

## イメージモザイク

イメージモザイク（モザイキング）は画像の幾何変換を用いて，画像をつなぎ合わせる処理である．
以下では，イメージモザイクにより，2枚の画像をつなぎ合わせる．

まず，イメージモザイクに使用する画像を読み込む．

In [ ]:
img1 = cv2.imread('./image1/Blackboard1.jpg')
img2 = cv2.imread('./image1/Blackboard2.jpg')
img3 = cv2.imread('./image1/Blackboard3.jpg')
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(8, 16))
plt.subplot(311),plt.imshow(img1),plt.title('img1')
plt.subplot(312),plt.imshow(img2),plt.title('img2')
plt.subplot(313),plt.imshow(img3),plt.title('img3')
plt.show()

#### 対応点の決定

次に，基準となる対応点を求めます．
今回は画像中の黒板の中で特徴的な部分を対応点として，それぞれの画像の対する座標を指定します．

In [ ]:
xy1 = np.array([[1378, 441],
                [1375, 532],
                [1540, 621],
                [1496, 446]])

xy2 = np.array([[ 953, 432],
                [ 950, 520],
                [1107, 607],
                [1066, 438]])

xy3 = np.array([[ 439, 421],
                [ 435, 512],
                [ 594, 600],
                [ 554, 431]])

指定した対応点を描画します．
同じ色のマーカーが同じ対応点を表しています．

In [ ]:
plt.figure(figsize=(8, 16))

plt.subplot(311)
plt.imshow(img1)
plt.scatter(xy1[:,0], xy1[:,1], marker='x', c=range(4), cmap='rainbow')

plt.subplot(312)
plt.imshow(img2)
plt.scatter(xy2[:,0], xy2[:,1], marker='x', c=range(4), cmap='rainbow')

plt.subplot(313)
plt.imshow(img3)
plt.scatter(xy3[:,0], xy3[:,1], marker='x', c=range(4), cmap='rainbow')

plt.show()

#### 幾何変換の推定

指定した対応点を用いて，幾何変換の推定を行います．
今回は，`img1`を基準として，`img3`を変換するための行列を求めます．

行列の推定には，OpenCVの`getPerspectiveTransform`関数を使用します．
`getPerspectiveTransform`関数では，4点の対応点から変換行列を計算します．

推定した行列`P`とscikit-imageの`warp`関数を用いて，画像を変換します．

その後，変換後の画像と基準となる`img1`をつなぎ合わせ，1枚の画像とします．


In [ ]:
from skimage.transform import warp

# 変換行列の計算
P = cv2.getPerspectiveTransform(xy1.astype(np.float32), xy3.astype(np.float32))

# 画像の変換
f_stitched = warp(img3, P, output_shape=(1200, 2800))
f_stitched = f_stitched * 255
f_stitched = f_stitched.astype(np.uint8)
plt.figure(figsize=(12, 6))
plt.imshow(f_stitched)
plt.show()

# 変換した画像と基準となる画像を合成する
h, w = img1.shape[:2]
f_stitched[0:h, 0:w, :] = img1
plt.figure(figsize=(12, 6))
plt.imshow(f_stitched)
plt.show()

## 課題

* 各座標変換の変数を変更し，どのように変化するか確認すること
* イメージモザイキングの対応点の一部や使用する画像（`img2`）を変更すると，合成結果がどのように変化するか確認すること